In [2]:
import sys
sys.path.insert(0, '../../')  
import numpy as np
import matplotlib.pyplot as plt
from gasp import simulation, responses, ssfp, gasp
from skimage.metrics import structural_similarity as ssim
from scipy.stats import pearsonr

def calculate_metrics(original, noisy):
    """Calculate various noise metrics."""
    mse = np.mean((original - noisy) ** 2)
    rmse = np.sqrt(mse)
    max_signal = np.max(original)
    snr = np.mean(original) / np.std(noisy - original)
    psnr = 20 * np.log10(max_signal) - 10 * np.log10(mse)
    ssim_value = ssim(original, noisy, data_range=max_signal)
    nrmse = rmse / (np.max(original) - np.min(original))
    spectral_fidelity, _ = pearsonr(original.flatten(), noisy.flatten())
    return {
        'SNR': snr,
        'PSNR': psnr,
        'SSIM': ssim_value,
        'NRMSE': nrmse,
        'Spectral Fidelity': spectral_fidelity
    }

def gasp_noise_analysis(noise_levels, off_resonance_levels, filter_types):
    # Simulation parameters
    width, height = 256, 256
    npcs = 16
    TRs = [5e-3, 10e-3, 20e-3]
    alpha = np.deg2rad(60)
    gradient = 2 * np.pi

    # Results storage
    results = {filter_type: {metric: {'noise': [], 'off_res': []} 
                             for metric in ['SNR', 'PSNR', 'SSIM', 'NRMSE', 'Spectral Fidelity']}
               for filter_type in filter_types}

    for filter_type in filter_types:
        print(f"Analyzing filter: {filter_type}")
        
        # Generate filter response
        if filter_type == 'gaussian':
            D = responses.gaussian(width, 0.2, 0)
        elif filter_type == 'square':
            D = responses.square(width, 0.2, 0)
        elif filter_type == 'sinc':
            D = responses.sinc(width, 0.2, 0)
        
        # Simulate SSFP data
        M = simulation.simulate_ssfp(width=width, height=height, npcs=npcs, TRs=TRs, alpha=alpha, gradient=gradient)
        
        # Train GASP model
        Ic, An = simulation.train_gasp(M, D)
        
        # Noise analysis
        for noise_level in noise_levels:
            M_noisy = ssfp.add_noise(M, sigma=noise_level)
            Ic_noisy = gasp.apply_gasp(M_noisy, An)
            metrics = calculate_metrics(np.abs(Ic), np.abs(Ic_noisy))
            for metric, value in metrics.items():
                results[filter_type][metric]['noise'].append(value)
        
        # Off-resonance analysis
        for off_res_level in off_resonance_levels:
            M_off_res = simulation.simulate_ssfp(width=width, height=height, npcs=npcs, TRs=TRs, alpha=alpha, gradient=gradient + off_res_level)
            Ic_off_res = gasp.apply_gasp(M_off_res, An)
            metrics = calculate_metrics(np.abs(Ic), np.abs(Ic_off_res))
            for metric, value in metrics.items():
                results[filter_type][metric]['off_res'].append(value)

    return results

def plot_results(results, noise_levels, off_resonance_levels):
    metrics = list(next(iter(results.values())).keys())
    fig, axes = plt.subplots(len(metrics), 2, figsize=(15, 5*len(metrics)))

    for i, metric in enumerate(metrics):
        for j, (x, xlabel, key) in enumerate([
            (noise_levels, 'Noise Level (σ)', 'noise'),
            (off_resonance_levels, 'Off-Resonance Level (rad/s)', 'off_res')
        ]):
            ax = axes[i, j]
            for filter_type, data in results.items():
                ax.plot(x, data[metric][key], label=filter_type, marker='o')
            ax.set_xlabel(xlabel)
            ax.set_ylabel(metric)
            ax.set_title(f'{metric} vs {xlabel}')
            ax.legend()
            ax.grid(True)

    plt.tight_layout()
    plt.show()

# Run the analysis
noise_levels = np.linspace(0.001, 0.1, 10)
off_resonance_levels = np.linspace(0, 1, 10)
filter_types = ['gaussian', 'square', 'sinc']

results = gasp_noise_analysis(noise_levels, off_resonance_levels, filter_types)
plot_results(results, noise_levels, off_resonance_levels)

Analyzing filter: gaussian


ValueError: shapes (256,256,1,16,3) and (1,48) not aligned: 3 (dim 4) != 1 (dim 0)